# Using native Python

## Default local client

When rgxlog is imported, a default client is created behind the scenes. This is the client that %%spanner uses.
However, one can also use his own client in the following manner:

Using a client manually enables one to dynamically generate querries facts and rules

In [1]:
import rgxlog

/home/dean/miniconda3/envs/bench/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
2021-01-27 18:02:07,825 - port 32768 is already taken
2021-01-27 18:02:07,828 - client connected to localhost:32769


In [2]:
from rgxlog import Client

client = Client()
# Notice the default port is taken by the default client, hence the next port is used for the user's client

2021-01-27 18:02:07,838 - port 32768 is already taken
2021-01-27 18:02:07,841 - port 32769 is already taken
2021-01-27 18:02:07,842 - listening on port 32770
2021-01-27 18:02:07,845 - client connected to localhost:32770
2021-01-27 18:02:07,845 - listener accepted connection from ('127.0.0.1', 45986)
2021-01-27 18:02:16,613 - listener connection closed


In [3]:
result = client.execute('''
    new uncle(str, str)
    uncle("bob", "greg")
    ''')
print(result)

In [4]:
for maybe_uncle in ['bob','greg','jhon']:
    result = client.execute(f'''
    ?uncle("{maybe_uncle}",Y)
    ''')
    print(result)

printing results for query 'uncle("bob", Y)':
  Y
------
 greg

printing results for query 'uncle("greg", Y)':
[]

printing results for query 'uncle("jhon", Y)':
[]



In [5]:
# client = None  # Notice the connection closes, due to GC
# client.disconnect()  # if you wish to disconnect explicitly you can do so as well

## Manually configured remote client

The following cell demonstrates connecting to a remote server.
* Start the remote server (run server.py)
* Note down the server's port and ip
* Use the following code snippet with your port and ip to connect

In [ ]:
from rgxlog import Client
# the server can also be remote

# this will fail unless you have a RGXlog server running on this ip and port
custom_client = Client(
    remote_ip='192.168.0.100',
    remote_port=32678
)

result = custom_client.execute('''
    new uncle(str, str)
    uncle("bob", "greg")
    ?uncle(X,Y)
    ''')
print(result)

custom_client.disconnect()

# Changing the client of the magic cells

In cases where you want to work with a custom client, but still make use of the magic system, you can overide the client used by the magic system

In [6]:
import rgxlog  # default client starts here, and captures 32768
from rgxlog import Client

another_client=Client()  # therefore this client will try the next in line, 32769

2021-01-27 18:02:11,073 - port 32768 is already taken
2021-01-27 18:02:11,109 - port 32769 is already taken
2021-01-27 18:02:11,113 - port 32770 is already taken
2021-01-27 18:02:11,116 - listening on port 32771
2021-01-27 18:02:11,121 - client connected to localhost:32771
2021-01-27 18:02:11,125 - listener accepted connection from ('127.0.0.1', 42598)
2021-01-27 18:02:12,647 - listener connection closed


In [7]:
%%spanner
# still using the default client

new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

printing results for query 'uncle(X, Y)':
  X  |  Y
-----+------
 bob | greg



In [8]:
default_client = rgxlog.magic_client  # save the default for later (prevents GC)
rgxlog.magic_client = another_client

In [9]:
%%spanner
# using 'another_client', not the default

new uncle2(str, str)
uncle2("bob", "greg")
?uncle(X,Y)

exception during semantic checks or execution:
 relation "uncle" is not defined


In [10]:
another_client.disconnect()
rgxlog.magic_client = default_client

2021-01-27 18:02:12,648 - client connection closed


In [11]:
%%spanner
# back to using the default client
new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

exception during semantic checks or execution:
 relation "uncle" is already defined. relation redefinitions are not allowed


# Mixing magics with dynamic client calls

Lets take the GPA example from the introductory tutorial.
What if we want to have multiple rules each looking for GPAs of students in different classes.
We wouldnt want to manually write a rule for every single subject.

In [12]:
import rgxlog

In [13]:
client=rgxlog.magic_client

2021-01-27 18:02:16,614 - client connection closed


We write our data manually. In the future we would be able to import it from csvs/dataframes

In [14]:
%%spanner
new lecturer(str, str)
lecturer("walter", "chemistry")
lecturer("linus", "operation_systems")
lecturer("rick", "physics")

new enrolled(str, str)
enrolled("abigail", "chemistry")
enrolled("abigail", "operation_systems")
enrolled("jordan", "chemistry")
enrolled("gale", "operation_systems")
enrolled("howard", "chemistry")
enrolled("howard", "physics")



gpa_str = "abigail 100 jordan 80 gale 79 howard 60"


In [15]:
%%spanner

gpa(Student,Grade) <- RGXString(gpa_str, "(\w+).*?(\d+)")->(Student, Grade),enrolled(Student,X)

?gpa(X,Y)

printing results for query 'gpa(X, Y)':
    X    |   Y
---------+-----
 howard  |  60
  gale   |  79
 jordan  |  80
 abigail | 100



Now we are going to define the rules using a for loop

In [16]:
subjects=[
    "chemistry",
    "physics",
    "operation_systems",
    "magic",
]

for subject in subjects:
    rule=f"""
    gpa_of_{subject}_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "{subject}")
    """
    client.execute(rule)
    print(rule) # we print the rule here to show you what strings are sent to the client


    gpa_of_chemistry_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "chemistry")
    

    gpa_of_physics_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "physics")
    

    gpa_of_operation_systems_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "operation_systems")
    

    gpa_of_magic_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "magic")
    


As you can see, we can use the dynamically defined rules in a magic cell

In [17]:
%%spanner
?gpa_of_operation_systems_students(X,Y)

printing results for query 'gpa_of_operation_systems_students(X, Y)':
    X    |   Y
---------+-----
  gale   |  79
 abigail | 100



And we can also query dynamically

In [18]:
subjects=[
    "chemistry",
    "physics",
    "operation_systems",
    "magic",
]

for subject in subjects:
    query=f"""
    ?gpa_of_{subject}_students(Student, Grade)
    """
    result=client.execute(query)
    print(result)

printing results for query 'gpa_of_chemistry_students(Student, Grade)':
  Student  |   Grade
-----------+---------
  howard   |      60
  jordan   |      80
  abigail  |     100

printing results for query 'gpa_of_physics_students(Student, Grade)':
  Student  |   Grade
-----------+---------
  howard   |      60

printing results for query 'gpa_of_operation_systems_students(Student, Grade)':
  Student  |   Grade
-----------+---------
   gale    |      79
  abigail  |     100

printing results for query 'gpa_of_magic_students(Student, Grade)':
[]



# Processing the result of a query in python and using the result in a new query

In [19]:
from rgxlog import Client

In [20]:
client = Client()
result = client.execute(f'''
    new friends(str, str, str)
    friends("bob", "greg", "clyde")
    friends("steven", "benny", "horace")
    friends("lenny", "homer", "toby")
    ?friends(X,Y,Z)
    ''')
print(result)
# Notice the default port is taken by the default client, hence the next port is used for the user's client


# now we'll showcase processing the result with native python...
result = result.split('\n')[3:-1]
result_tuples = []
for row in result:
    result_tuples.append(tuple(map(str.strip, row.split('|'))))
# future versions will do this internally

# lets filter our tuples with some predicate:
filtered = tuple(filter(lambda friends: 'bob' in friends or 'lenny' in friends, result_tuples))

# and feed the matching tuples into a new query:
client.execute('new buddies(str, str)')

for first, second, _ in filtered:
    client.execute(f'buddies("{first}", "{second}")')

result = client.execute("?buddies(First, Second)")
print(result)

2021-01-27 18:02:42,204 - port 32768 is already taken
2021-01-27 18:02:42,227 - port 32769 is already taken
2021-01-27 18:02:42,228 - listening on port 32770
2021-01-27 18:02:42,230 - client connected to localhost:32770
2021-01-27 18:02:42,230 - listener accepted connection from ('127.0.0.1', 46016)


printing results for query 'friends(X, Y, Z)':
   X    |   Y   |   Z
--------+-------+--------
 lenny  | homer |  toby
 steven | benny | horace
  bob   | greg  | clyde

printing results for query 'buddies(First, Second)':
  First  |  Second
---------+----------
   bob   |   greg
  lenny  |  homer



# Developing and using your own IE functions

To make your own ie function you need to make a new python file and place it in rgxlog/user_ie_functions.
In the python file you need to create a new class that inherits from IEFunctionData.
By implementing its methods you are suppling the input format, output format, and the function itself.

Note that IE functions are supplied by an IEFunctionData subclass with 3 main elements:
Methods for type checking:
* a `get_input_types` method returns the DataTypes of the relation that the ie function expects
* a `get_output_types` method returns the DataTypes of the relation that the ie function will return.
    * This function takes as a parameter the output arity defined in the RGXlog statement.
    * This is done to partially enable template ie functions that can return multiple arities, like RGX log
And the ie function itself
* the `ie_function
    * Which takes a typle of `get_input_type` 
    * and returns an iterator of tuples of `get_output_types`

In [21]:
# example:

import spacy
sp = spacy.load('en_core_web_sm')

from rgxlog.engine.datatypes.primitive_types import DataTypes
from rgxlog.engine.ie_functions.ie_function_base import IEFunctionData

class Entities(IEFunctionData):
    def __init__(self):
        super().__init__()

    @staticmethod
    def ie_function(text):  # this is where you implement the actual processing
        entities = sp(text).ents
        return ((entity.text, spacy.explain(entity.label_)) for entity in entities)

    @staticmethod
    def get_input_types():  # this is where you define the inputs
        return [DataTypes.string]

    @staticmethod
    def get_output_types(output_arity):  # and this is where you define your outputs
        return tuple([DataTypes.string] * 2)

# place this file in rgxlog/user_ie_functions

next, we need to register the function by using the class name:

In [22]:
import rgxlog
rgxlog.magic_client.register('Entities')

'registered Entities'

now we're free to use it:

In [23]:
%%spanner
text =  "You've been living in a dream world, Neo.\
        As in Baudrillard's vision, your whole life has been spent inside the map, not the territory.\
        This is the world as it exists today.\
        Welcome to the desert of the real."
entities(Entity, Classification) <- Entities(text)->(Entity, Classification)
?entities(Entity, Classification)

printing results for query 'entities(Entity, Classification)':
   Entity    |             Classification
-------------+-----------------------------------------
    today    |  Absolute or relative dates or periods
 Baudrillard | Companies, agencies, institutions, etc.
     Neo     | Companies, agencies, institutions, etc.

